# Level 4.5: Agentic RAG with reference and LM-Eval Eval

This tutorial presents an example of evaluating an agentic RAG in LLama-Stack using the reference implementation and a custom
provider using the [Language Model Evaluation Harness](https://github.com/EleutherAI/lm-evaluation-harness) tool,
aka `LM-Eval`. 

Please refer to `# Level4_agentic_RAG.ipynb` [notebook](../rag_agentic/notebooks/Level4_RAG_agent.ipynb)
for details on how to initialize the agent and the knowledge search RAG tool provided by Llama Stack.

## Overview

This tutorial covers the following steps:
1. Connecting to a llama-stack server.
2. Indexing a collection of documents in a vector DB for later retrieval.
3. Initializing the agent capable of retrieving content from vector DB via tool use.
4. Evaluating the agent responses against a reference set of Q&A.
5. Reporting the evaluation results and its statistical relevance.

## Case study
For the purpose of this training, we are going to use the fictional company 
[Parasol Financial](https://www.redhat.com/en/blog/ai-insurance-industry-insights-red-hat-summit-2024), and the provided
[training documents](https://github.com/jharmison-redhat/parasol-financial-data/).

A sample Q&A document is available as a [reference](./data/parasol-financial-data_qac.yaml). 
This predefined question and answer pairs have beeen generated using [docling-sdg](https://github.com/docling-project/docling-sdg),
an IBM set of tools to create artificial data from documents, leveraging generative AI and Docling's parsing capabilities.

## Prerequisites

Before starting, ensure you have a running instance of the Llama Stack server (local or remote) with at least one preconfigured vector DB. For more information, please refer to the corresponding [Llama Stack tutorials](https://llama-stack.readthedocs.io/en/latest/getting_started/index.html).

The `openai` inference provider is required if you intend to use an OpenAI model for judging purposes, like `openai/gpt-4o`. In this case, the 
`OPENAI_API_KEY` env variable must be configured into the Llama Stack server.

**Notes**:
* In order to run the evaluation steps with the Llama Stack reference implementation, the recommended deployment is the one 
  available in `kubernetes/kustomize/overlay/eval`.
* To run the evaluation steps with the LM-Eval implementation, the recommended deployment is the one 
  available in `kubernetes/kustomize/overlay/lmeval` (which also includes the above requirements).

## Setting the Environment Variables

Use the [`.env.example`](../../../.env.example) to create a new file called `.env` and ensure you add all the relevant environment variables below.

In addition to the environment variables listed in the ["Getting Started" notebook](../rag_agentic/notebooks/Level0_getting_started_with_Llama_Stack.ipynb), the following should be provided for this demo to run:
 - `LLM_AS_JUDGE_MODEL_ID`: the model to use as the judge to evaluate the agent responses. Must be one of the models defined in Llama Stack.
 - `VDB_PROVIDER`: the vector DB provider to be used. Must be supported by Llama Stack. For this demo, we use Milvus Lite which is our preferred solution.
 - `VDB_EMBEDDING`: the embedding model to be used for ingestion and retrieval. For this demo, we use all-MiniLM-L6-v2.
 - `VDB_EMBEDDING_DIMENSION` (optional): the dimension of the embedding. Defaults to 384.
 - `VECTOR_DB_CHUNK_SIZE` (optional): the chunk size for the vector DB. Defaults to 512.

## 1. Setting Up the Environment
We will start with a few imports needed for this demo only.

In [76]:
import numpy as np
import pandas as pd
import time
import uuid

from rich.pretty import pprint

from IPython.display import display_markdown

from llama_stack_client import Agent, AgentEventLogger, RAGDocument
from llama_stack_client.lib.agents.event_logger import EventLogger

Next, we will initialize our environment as described in detail in our ["Getting Started" notebook](../rag_agentic/notebooks/Level0_getting_started_with_Llama_Stack.ipynb). Please refer to it for additional explanations.

In [77]:
# for accessing the environment variables
import os
from dotenv import load_dotenv
load_dotenv(override=True)

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient
# to override the judge model
from llama_stack.providers.inline.scoring.llm_as_judge.scoring_fn.fn_defs.llm_as_judge_405b_simpleqa import (
    llm_as_judge_405b_simpleqa,
)

# pretty print of the results returned from the model/agent
import sys
sys.path.append('..')  
from rag_agentic.src.utils import step_printer
from termcolor import cprint

remote = os.getenv("REMOTE", "True")

if remote == "False":
    local_port = os.getenv("LOCAL_SERVER_PORT", 8321)
    base_url = f"http://localhost:{local_port}"
else: # any value non equal to 'False' will be considered as 'True'
    base_url = os.getenv("REMOTE_BASE_URL")

client = LlamaStackClient(
    base_url=base_url,
    provider_data=None
)
    
print(f"Connected to Llama Stack server @ {base_url}")

# model_id will later be used to pass the name of the desired inference model to Llama Stack Agents/Inference APIs
model_id = os.getenv("INFERENCE_MODEL_ID")

temperature = float(os.getenv("TEMPERATURE", 0.0))
if temperature > 0.0:
    top_p = float(os.getenv("TOP_P", 0.95))
    strategy = {"type": "top_p", "temperature": temperature, "top_p": top_p}
else:
    strategy = {"type": "greedy"}

max_tokens = int(os.getenv("MAX_TOKENS", 4096))

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}

stream_env = os.getenv("STREAM", "True")
# the Boolean 'stream' parameter will later be passed to Llama Stack Agents/Inference APIs
# any value non equal to 'False' will be considered as 'True'
stream = (stream_env != "False")

# The Q&A file
QNA_FILE = './data/parasol-financial-data_qac.yaml'
# The number of rows to consider
MAX_QNA_ROWS = 50
# Set to True to enable display of evaluation results
EVAL_DEBUG = False
llm_as_judge_model = os.getenv("LLM_AS_JUDGE_MODEL_ID")
llm_as_judge_405b_simpleqa_params = llm_as_judge_405b_simpleqa.params.model_copy()
# Override the default model
# To update the scoring params, we need to provide all the settings, including the defaults
llm_as_judge_405b_simpleqa_params.judge_model = llm_as_judge_model

# Convert the model dump to a dictionary
scoring_params = llm_as_judge_405b_simpleqa_params.model_dump()
scoring_params['aggregation_functions']=['categorical_count']

print(f"Inference Parameters:\n\tModel: {model_id}\n\tSampling Parameters: {sampling_params}\n\tstream: {stream}")
print(f"Eval Parameters:\n\tJudge Model: {llm_as_judge_model}\n\tQ&A file: {QNA_FILE}\n\tMax rows: {MAX_QNA_ROWS}")

Connected to Llama Stack server @ http://localhost:8321
Inference Parameters:
	Model: granite32-8b
	Sampling Parameters: {'strategy': {'type': 'greedy'}, 'max_tokens': 4096}
	stream: True
Eval Parameters:
	Judge Model: openai/gpt-4o
	Q&A file: ./data/parasol-financial-data_qac.yaml
	Max rows: 50


Finally, we will initialize the document collection to be used for RAG ingestion and retrieval.

In [78]:
vector_db_id = f"test_vector_db_{uuid.uuid4()}"
display_markdown(f"Registered vector DB **{vector_db_id}**", raw=True)


Registered vector DB **test_vector_db_dc9b592d-e12f-4720-8fed-5fedd596ffbd**

## 2. Indexing the Documents
- Initialize a new document collection in the target vector DB. All parameters related to the vector DB, such as the embedding model and dimension, must be specified here.
- Provide a list of document URLs to the RAG tool. Llama Stack will handle fetching, conversion and chunking of the documents' content.
- Perform a sample query to verify the response is retrieved from the relevant documents.

In [79]:
# define and register the document collection to be used
client.vector_dbs.register(
    vector_db_id=vector_db_id,
    embedding_model=os.getenv("VDB_EMBEDDING"),
    embedding_dimension=int(os.getenv("VDB_EMBEDDING_DIMENSION", 384)),
    provider_id=os.getenv("VDB_PROVIDER"),
)

# ingest the documents into the newly created document collection
urls = [
    "flexible_enhanced_checking/flexible_enhanced_checking.md",
    "flexible_savings/flexible_savings.md",
    "flexible_premier_checking/flexible_premier_checking.md",
    "flexible_core_checking/flexible_core_checking.md",
    "policies/online_service_agreement.md",
    "enablement/customer_interactions_resource_guide.md",
    "enablement/banking_essentials_resource_guide.md",
    "flexible_money_market_savings/flexible_money_market_savings.md",
    "flexible_checking/flexible_checking.md",
]
documents = [
    RAGDocument(
        document_id=f"{url.split('/')[-1]}",
        content=f"https://raw.githubusercontent.com/jharmison-redhat/parasol-financial-data/main/{url}",
        mime_type="text/plain",
        metadata={},
    )
    for i, url in enumerate(urls)
]
client.tool_runtime.rag_tool.insert(
    documents=documents,
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=int(os.getenv("VECTOR_DB_CHUNK_SIZE", 512)),
)

In [82]:
# Query documents
results = client.tool_runtime.rag_tool.query(
    vector_db_ids=[vector_db_id],
    content="What is the Parasol Financial Withdrawal Limit Fee and Transaction Limitations for Flexible Money Market Savings",
)
results.metadata['document_ids']

['flexible_money_market_savings.md',
 'flexible_money_market_savings.md',
 'flexible_savings.md',
 'flexible_savings.md',
 'online_service_agreement.md']

## 3. Defining reusable functions
Define reusable Python functions to use during the execution of the evaluation jobs.


In [83]:
def accuracy_from_categorical_count(response):
    """
    Computes the evaluation accuracy from the responses of the `llm-as-judge::405b-simpleqa`
    scoring function.

    Expected responses are:
    ```
    A: CORRECT
    B: INCORRECT
    C: NOT_ATTEMPTED
    ```
    The accuracy is computed as: <number of responses of type `A`> / <number of responses> * 100
    """
    # Evaluate numerical score
    correct_answers = sum(
        [
            count
            for cat, count in response.scores["llm-as-judge::405b-simpleqa"]
            .aggregated_results["categorical_count"]["categorical_count"]
            .items()
            if cat == "A"
        ]
    )
    num_of_scores = len(response.scores["llm-as-judge::405b-simpleqa"].score_rows)
    return correct_answers / num_of_scores * 100

In [84]:
def _run_eval(use_rag: bool):
    """
    Runs the evaluation function for the benchmark indicated by the global variable `qna_benchmark_id`.
    A new agent is created for every function call: in case `use_rag` is set to `True`, the `knowledge_search` tool is defined
    to implement the RAG workflow.
    The global variables `model_id` and `vector_db_id` are also requested.

    Params:
        use_rag: whether to run a RAG workflow or not.
    Returns:
        the `Job` associated to the evaluation function.
    """

    from httpx import Timeout

    if use_rag == True:
        instructions = "You are a helpful assistant. You must use the knowledge search tool to answer user questions."
        tools = [
            dict(
                name="builtin::rag",
                args={
                    "vector_db_ids": [
                        vector_db_id
                    ],  # list of IDs of document collections to consider during retrieval
                },
            )
        ]
    else:
        instructions = "You are a helpful assistant."
        tools = []

    agent_config = {
        "model": model_id,
        "instructions": instructions,
        "sampling_params": sampling_params,
        "toolgroups": tools,
    }

    _job = client.eval.run_eval(
        benchmark_id=qna_benchmark_id,
        benchmark_config={
            "num_examples": MAX_QNA_ROWS,
            "scoring_params": {
                "llm-as-judge::405b-simpleqa": scoring_params,
            },
            "eval_candidate": {
                "type": "agent",
                "config": agent_config,
            },
        },
        timeout=Timeout(MAX_QNA_ROWS * 30),  # Allow for 30s per Q&A
    )
    return _job

In [85]:
def get_job_status(job_id, benchmark_id):
    return client.eval.jobs.status(job_id=job_id, benchmark_id=benchmark_id).status

def _get_eval_reponse(_job):
    """
    Returns the `EvalResponse` instance for the given `_job`.

    Params:
        `job_id`: The evaluation `Job`.
    Returns:
        The `EvalResponse` for the given `_job`
    """
    status = get_job_status(
        benchmark_id=qna_benchmark_id, job_id=_job.job_id
    )
    while status != "completed":
        print(f"Job status is {status}")
        sleep(1)
        status = client.eval.jobs.status(
            benchmark_id=qna_benchmark_id, job_id=_job.job_id
        ).status
    print(f"Job status is {status}")
    _eval_response = client.eval.jobs.retrieve(
        benchmark_id=qna_benchmark_id, job_id=_job.job_id
    )

    return _eval_response

In [86]:
def to_label(score_row):
    """
    Returns the display label for the given `score_row`.
    """
    grades = {'A': 'CORRECT', 'B': 'INCORRECT', 'C': 'NOT_ATTEMPTED'}
    score = score_row.get('score', str(score_row))
    return grades.get(score,  f'UNKNOWN {score}')

In [87]:
def numeric_scores(response):
    """
    Converts the computed scores in a numeric array, where scores `A` are evaluated to 1
    and all the others to `0`.
    """
    def category_to_number(category):
        if category == 'A':
            return 1
        return 0

    return [category_to_number(score_row['score']) for score_row in response.scores['llm-as-judge::405b-simpleqa'].score_rows]

In [88]:
def permutation_test_for_paired_samples(scores_a, scores_b, iterations=10_000):
    """
    Performs a permutation test of a given statistic on provided data.
    """

    from scipy.stats import permutation_test


    def _statistic(x, y, axis):
        return np.mean(x, axis=axis) - np.mean(y, axis=axis)

    result = permutation_test(
        data=(scores_a, scores_b),
        statistic=_statistic,
        n_resamples=iterations,
        alternative='two-sided',
        permutation_type='samples'
    )
    return float(result.pvalue)

In [89]:
def print_stats_significance(scores_a, scores_b, label_a, label_b):
    mean_score_a = np.mean(scores_a)
    mean_score_b = np.mean(scores_b)

    p_value = permutation_test_for_paired_samples(scores_a, scores_b)
    print(model_id)
    print(f" {label_a:<50}: {mean_score_a:>10.4f}")
    print(f" {label_b:<50}: {mean_score_b:>10.4f}")
    print(f" {'p_value':<50}: {p_value:>10.4f}")
    print()

    if p_value < 0.05:
        print("p_value<0.05 so this result is statistically significant")
        # Note that the logic below if wrong if the mean scores are equal, but that can't be true if p<1.
        higher_model_id = (
            label_a
            if mean_score_a >= mean_score_b
            else label_b
        )
        print(f"You can conclude that {higher_model_id} generation is better on data of this sort")
    else:
        import math

        print("p_value>=0.05 so this result is NOT statistically significant.")
        print(
            f"You can conclude that there is not enough data to tell which is better."
        )
        num_samples = len(scores_a)
        margin_of_error = 1 / math.sqrt(num_samples)
        print(
            f"Note that this data includes {num_samples} questions which typically produces a margin of error of around +/-{margin_of_error:.1%}."
        )
        print(f"So the two are probably roughly within that margin of error or so.")

## 4. Creating an evaluation Dataset
- Load the Q&A file as a Pandas DataFrame.
- Transform the dataset to a schema suitable for LLS evaluations.
- Register a new Dataset.

In [90]:
with open(QNA_FILE, "r") as f:
    qnas_df = pd.read_json(f, lines=True)
pd.set_option("display.max_colwidth", None)

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [91]:
from llama_stack.apis.inference import UserMessage
import json
import random

qna_dataset_rows = []

chat_completion_input = UserMessage(content="")
for i in range(len(qnas_df)):
    qna = {}
    qna["input_query"] = qnas_df.iloc[i]["question"]
    qna["expected_answer"] = qnas_df.iloc[i]["answer"]

    chat_completion_input.content = qna["input_query"]
    qna["chat_completion_input"] = json.dumps([chat_completion_input.model_dump()])

    qna_dataset_rows.append(qna)

random.shuffle(qna_dataset_rows)
qna_dataset_df = pd.DataFrame(qna_dataset_rows)
qna_dataset_df.head()

,input_query,expected_answer,chat_completion_input
0,What methods may be used to communicate changes to the Agreement?,"Changes may be communicated by mail, email, or a notice on the website.","[{""role"": ""user"", ""content"": ""What methods may be used to communicate changes to the Agreement?"", ""context"": null}]"
1,Why might it be important to resolve a customer's issue the first time they contact you?,"Resolving a customer's issue on the first contact is important because it ensures high-quality service, addresses customer concerns efficiently, and provides the right resolution, which can lead to increased customer satisfaction and loyalty.","[{""role"": ""user"", ""content"": ""Why might it be important to resolve a customer's issue the first time they contact you?"", ""context"": null}]"
2,What type of fees might you incur if your available balance is not sufficient to process scheduled payments or transfers?,Returned item fees from the payee or overdraft fees.,"[{""role"": ""user"", ""content"": ""What type of fees might you incur if your available balance is not sufficient to process scheduled payments or transfers?"", ""context"": null}]"
3,What are the benefits of acting with empathy towards customers?,"The benefits of acting with empathy towards customers include gaining their trust, engaging in better conversations, encouraging them to open up about their life, needs, and goals, having them listen to your point of view, being considered the next time they have a need, being referred to others, and agreeing to connect with you again.","[{""role"": ""user"", ""content"": ""What are the benefits of acting with empathy towards customers?"", ""context"": null}]"
4,Who is the publisher of the resource guide on financial services?,Parasol Financial Corporation.,"[{""role"": ""user"", ""content"": ""Who is the publisher of the resource guide on financial services?"", ""context"": null}]"


In [92]:
qna_dataset_id = f"test_dataset_{uuid.uuid1()}"
_ = client.datasets.register(
    purpose="eval/messages-answer",
    source={
        "type": "rows",
        "rows": qna_dataset_rows,
    },
    dataset_id=qna_dataset_id,
)
display_markdown(f"Registered dataset **{qna_dataset_id}**", raw=True)


Registered dataset **test_dataset_04b8b2fa-3622-11f0-b2c2-4a70c355aff9**

## 5. LLM Eval with reference implementation

### 5.1 Registering the evaluation benchmark
- Register a Benchmark using the Dataset and the `llm-as-judge::405b-simpleqa` scoring function.
- The benchmark is connected to the provider by the `provider_id` field.

In [93]:
qna_benchmark_id = f"test_benchmark_{uuid.uuid1()}"
client.benchmarks.register(
    provider_id="meta-reference",
    benchmark_id=qna_benchmark_id,
    dataset_id=qna_dataset_id,
    scoring_functions=["llm-as-judge::405b-simpleqa"],
)
display_markdown(f"Registered benchmark **{qna_benchmark_id}**", raw=True)


Registered benchmark **test_benchmark_3d97a338-3622-11f0-b2c2-4a70c355aff9**

### 5.2 LLM Eval without RAG
- Create an agent configuration without the `knowledge_search` tool.
- Run the evaluation function with the current configuration.

In [95]:
MAX_QNA_ROWS=5

In [96]:
start = time.time()

without_rag_responses = {}
_job = _run_eval(use_rag=False)
# pprint(_job)
_eval_response = _get_eval_reponse(_job)
if EVAL_DEBUG == True:
    pprint(_eval_response)

print(
    f"Evaluation of {MAX_QNA_ROWS} Q&A workflows completed in {time.time() - start:.3f} seconds"
)
display_markdown(
    f"**Computed accuracy is {accuracy_from_categorical_count(_eval_response)}%**",
    raw=True,
)
without_rag_responses[model_id] = _eval_response

Job status is completed
Evaluation of 5 Q&A workflows completed in 30.103 seconds


**Computed accuracy is 80.0%**

### 5.3. LLM Eval with RAG
- Create an agent configuration with the `knowledge_search` tool.
- Run the evaluation function with the current configuration.

In [97]:
start = time.time()

rag_responses = {}
_job = _run_eval(use_rag=True)
# pprint(_job)
_eval_response = _get_eval_reponse(_job)
if EVAL_DEBUG == True:
    pprint(_eval_response)

print(
    f"Evaluation of {MAX_QNA_ROWS} Q&A workflows completed in {time.time() - start:.3f} seconds"
)
display_markdown(
    f"**Computed accuracy is {accuracy_from_categorical_count(_eval_response)}%**",
    raw=True,
)
rag_responses[model_id] = _eval_response

retrieved_contexts = sum(
    [1 for r in rag_responses[model_id].generations if "context" in r]
)
display_markdown(
    f"**RAG knowledge search tool used in {retrieved_contexts} of ({MAX_QNA_ROWS}) agentic calls**",
    raw=True,
)

Job status is completed
Evaluation of 5 Q&A workflows completed in 26.601 seconds


**Computed accuracy is 80.0%**

**RAG knowledge search tool used in 4 of (5) agentic calls**

## 6. LLM Eval with LM-Eval implementation
We will run benchmark evaluations on the same dataset via LM-Eval.

**Implementation notes**:
- The `remote::lmeval` provider is implemented using the [llama_stack_provider_lmeval](https://github.com/trustyai-explainability/llama-stack-provider-lmeval/tree/main/src/llama_stack_provider_lmeval) package.
- The provider is connected to the deployed Llama Stack server by the following configuration in the [build.yaml](../../distribution/build.yaml):
```yaml
...
external_providers_dir: ./providers.d
```
- This `eval` provider delegates the execution of the evaluation function to a separate Kubernetes job that is implemented by the [LMEvalJob](https://trustyai-explainability.github.io/trustyai-site/main/lm-eval-tutorial.html) custom resource. This option provides better scalability and performance to the Llama Stack server, by executing the resource consuming tasks on on-demand
jobs dedicated to LLM inference and evaluation.
- Finally, the job is implemented as a Custom Task defined in [TrustyAI LM-Eval Tasks](https://github.com/trustyai-explainability/lm-eval-tasks)

### 6.1 Data preparation
The Q&A document prepared for the reference implementation is not suitable for the LM-Eval task, we need to rename some fields
accordingly

In [32]:
lmeval_qna_dataset_df = qna_dataset_df.copy()
lmeval_qna_dataset_df.rename(columns={
    'input_query': 'user_input',
    'expected_answer': 'reference'
}, inplace=True)
lmeval_qna_dataset_df.drop('chat_completion_input', inplace=True, axis=1)
lmeval_qna_dataset_df.head()


,user_input,reference
0,Why might it be important to resolve a customer's issue the first time they contact you?,"Resolving a customer's issue on the first contact is important because it ensures high-quality service, addresses customer concerns efficiently, and provides the right resolution, which can lead to increased customer satisfaction and loyalty."
1,Why might Wealth Management Specialists need to partner with all Parasol partners to deliver a full breadth of solutions?,"Partnering with all Parasol partners allows Wealth Management Specialists to offer a comprehensive range of financial solutions, which can help in deepening client relationships and effectively servicing the financial advisor team’s client base."
2,What are the steps involved in managing SMS text alerts for the service?,"To manage SMS text alerts, you can text STOP to 50014 to stop the alerts. If you want to restore the alerts, you need to go to the Alerts Settings pages and reactivate them. For help, you can send the word HELP to 50014."
3,What team is included in the additional resources to help employees manage stress?,Life Event Services team
4,What are the available payment options for the owner and child of a Core Checking for Family Banking account?,"The available payment options are using a Debit Card, Online and Mobile Bill Pay, and Online or Mobile Banking transfers."


We save this to a separate file and copy it in a PVC object that will be mounted to the LM-Eval job. The `dataset-storage-pod` Pod,
created by the `lmeval` overlay deployment, is used as the initializer to feed the data into the job.

**Note**: we assume the `oc` CLI is available and logged into the target namespace.

In [129]:
lmeval_qna_dataset_df.to_json(
    f"data/lmeval_qna_dataset.jsonl",
    orient="records",
    lines=True,
)

!oc cp ./data/lmeval_qna_dataset.jsonl dataset-storage-pod:/data/upload_files/lmeval_qna_dataset.jsonl

### 6.2 Registering the evaluation benchmark
The LM-Eval benchmark includes information about the custom task to be executed, the inference environment and the PVC to be mounted.

In [165]:
# Important note: This part after the '::' must match the task ID in the task repo
lmeval_benchmark_id = "lmeval::dk-bench"
client.benchmarks.register(
    benchmark_id=lmeval_benchmark_id,
    dataset_id=lmeval_benchmark_id,
    provider_id="lmeval",
    scoring_functions=["string"],
    provider_benchmark_id="string",
    # provide the GH details of the task
    metadata={
        "custom_task": {
            "git": {
                "url": "https://github.com/trustyai-explainability/lm-eval-tasks.git",
                "branch": "main",
                "commit": "8220e2d73c187471acbe71659c98bccecfe77958",
                "path": "tasks/",
            }
        },
        # provide container environment variables
        "env": {
            # specify path to the DK-Bench dataset within the dataset-storage-pod
            "DK_BENCH_DATASET_PATH": "/opt/app-root/src/hf_home/lmeval_qna_dataset.jsonl",
            # specify judge model arguments
            "JUDGE_MODEL_URL": "https://api.openai.com/v1",
            "JUDGE_MODEL_NAME": llm_as_judge_model,
            "JUDGE_API_KEY": os.getenv("OPENAI_API_KEY"),
        },
        # specify PVC name of the PVC to be used for the container
        "input": {"storage": {"pvc": "lmeval-data"}},
    },
)
display_markdown(f"Registered benchmark **{lmeval_benchmark_id}**", raw=True)

Registered benchmark **lmeval::dk-bench**

### 6.3. LM-Eval without RAG
Launch the job from the Llama Stack client.


In [ ]:
start = time.time()

job = client.eval.run_eval(
    benchmark_id="lmeval::dk-bench",
    benchmark_config={
        "eval_candidate": {
            "type": "model",
            "model": model_id,
            "provider_id": "lmeval",
            "sampling_params": sampling_params,

        },
        "num_examples": MAX_QNA_ROWS
    },)

print(f"Starting job '{job.job_id}'")


Starting job 'lmeval-job-af388259-3991-48a8-9f35-a434b6ed36f0'


Wait until the job completes.

In [170]:
while True:
    status = get_job_status(job_id=job.job_id, benchmark_id="lmeval::dk-bench")

    if status in ['failed', 'completed']:
        print(f"Job ended with status: {status}")
        break

    time.sleep(20)


Job ended with status: failed


Retrieve the result from the job.

In [187]:
_eval_response = client.eval.jobs.retrieve(
    job_id=job.job_id,
    benchmark_id="lmeval::dk-bench")

if EVAL_DEBUG == True:
    pprint(_eval_response)

print(
    f"Evaluation of {MAX_QNA_ROWS} Q&A workflows completed in {time.time() - start:.3f} seconds"
)
display_markdown(
    f"**Computed accuracy is {accuracy_from_categorical_count(_eval_response)}%**",
    raw=True,
)
lmeval_without_rag_responses[model_id] = _eval_response

Evaluation of 5 Q&A workflows completed in 23079.225 seconds


KeyError: 'llm-as-judge::405b-simpleqa'

### 6.4. LM-Eval with RAG
**TODO**: Add the agent evaluation to LM-Eval

In [181]:
lmeval_with_rag_responses = {}
lmeval_with_rag_responses[model_id] = None

## 4. Reporting
- Aggregated accuracy.
- Individual scores and responses.
- Statistical Significance.

In [185]:
pd_responses = {}
pd_responses['questions'] = [qna_dataset_rows[i]['input_query'] for i in range(MAX_QNA_ROWS)]
pd_responses['expected'] = [qna_dataset_rows[i]['expected_answer'] for i in range(MAX_QNA_ROWS)]

pd_accuracies = {}
df_accuracies = pd.DataFrame.from_dict({
    'Model': without_rag_responses.keys(),
    '(reference) Accuracy without RAG': [accuracy_from_categorical_count(without_rag_responses[model_id]) for model_id in without_rag_responses.keys()],
    '(reference) Accuracy with RAG': [accuracy_from_categorical_count(rag_responses[model_id]) for model_id in rag_responses.keys()],
    '(lmeval) Accuracy without RAG': [accuracy_from_categorical_count(lmeval_without_rag_responses[model_id]) for model_id in without_rag_responses.keys()],
    '(lmeval) Accuracy with RAG': [accuracy_from_categorical_count(lmeval_rag_responses[model_id]) for model_id in rag_responses.keys()]})
df_accuracies.style.hide()

NameError: name 'lmeval_without_rag_responses' is not defined

In [99]:
report_data = {}
ratings_data = {}
responses_data = {}

report_data['Question'] = [qna_dataset_rows[i]['input_query'] for i in range(MAX_QNA_ROWS)]
ratings_data['Question'] = report_data['Question']
responses_data['Question'] = report_data['Question']
report_data['Expected Answer'] = [qna_dataset_rows[i]['expected_answer'] for i in range(MAX_QNA_ROWS)]
responses_data['Expected Answer'] = report_data['Expected Answer']
for model_id in without_rag_responses.keys():
    report_data[f'Rating without RAG'] = [to_label(score_row) for score_row in without_rag_responses[model_id].scores['llm-as-judge::405b-simpleqa'].score_rows]
    report_data[f'Answer without RAG'] = [g['generated_answer'] for g in without_rag_responses[model_id].generations]
    report_data[f'Rating with RAG'] = [to_label(score_row) for score_row in rag_responses[model_id].scores['llm-as-judge::405b-simpleqa'].score_rows]
    report_data[f'Answer with RAG'] = [g['generated_answer'] for g in rag_responses[model_id].generations]
    
    ratings_data[f'Rating without RAG'] = report_data[f'Rating without RAG']
    responses_data[f'Answer without RAG'] = report_data[f'Answer without RAG']
    ratings_data[f'with RAG RAG Rating'] = report_data[f'Rating with RAG']
    responses_data[f'Answer with RAG'] = report_data[f'Answer with RAG']

In [100]:
from IPython.display import HTML


display_markdown(f"**Response Comparison for model {model_id}**", raw=True)
report_df = pd.DataFrame.from_dict(report_data)
HTML(report_df.head().to_html(index=False))


**Response Comparison for model granite32-8b**

Question,Expected Answer,Rating without RAG,Answer without RAG,Rating with RAG,Answer with RAG
What methods may be used to communicate changes to the Agreement?,"Changes may be communicated by mail, email, or a notice on the website.",CORRECT,"1. Email Notifications: This is a common method where changes to the agreement are communicated via email. It's important to clearly state the nature of the changes and their effective date.\n\n2. In-App Notifications: If the agreement is digital and used within an application, in-app notifications can be an effective way to communicate changes.\n\n3. Postal Mail: For formal agreements, changes might be communicated via postal mail, especially if the agreement is legally binding and requires a physical signature for acceptance.\n\n4. Website Announcements: If the agreement is available online, changes can be posted on the website with a clear notice about the revision date.\n\n5. Telephone or Video Call: For critical changes, a direct communication method like a phone call or video conference might be used to ensure the recipient understands the changes.\n\n6. Physical Copies: For agreements that are in physical form, changes can be communicated by sending out revised copies.\n\n7. Legal Notices: For significant changes, legal notices might be published in newspapers or on public notice boards.\n\n8. Agreement Update Platforms: Some platforms allow for direct updates within the agreement document itself, with a clear record of changes and who made them.\n\n9. Training Sessions: For complex changes, training sessions or webinars might be organized to explain the changes and their implications.\n\n10. Acknowledgment Forms: After communicating the changes, an acknowledgment form might be sent to the recipient to confirm they have read, understood, and agreed to the new terms. \n\nRemember, it's crucial to ensure that the method of communication is accessible and understandable to all parties involved, and that there's a clear record of when and how the changes were communicated.",INCORRECT,"Based on the retrieved information, changes to the agreement can be communicated through the following methods:\n\n1. **Mail**: The document mentions that changes to the agreement may be communicated via mail.\n\n2. **Email**: Changes can also be communicated through email. This is a common method for businesses to notify customers of updates or changes to their terms of service or agreements.\n\n3. **Notice on the website**: The agreement states that changes will be posted on the company's website. This is a digital method that allows for immediate access to updates for users who regularly check the site.\n\n4. **Secure mailbox**: This is a more secure method of communication, possibly used for sensitive information.\n\nRemember, the agreement also mentions that by continuing to use the services after the date that changes are posted to the website, those changes will be effective for transactions made after that date. This implies that regularly checking the website for updates is a crucial method for staying informed about changes to the agreement."
Why might it be important to resolve a customer's issue the first time they contact you?,"Resolving a customer's issue on the first contact is important because it ensures high-quality service, addresses customer concerns efficiently, and provides the right resolution, which can lead to increased customer satisfaction and loyalty.",CORRECT,"Resolving a customer's issue the first time they contact you is crucial for several reasons:\n\n1. **Customer Satisfaction**: Customers appreciate quick and efficient service. If their issue is resolved promptly, they're likely to feel satisfied and valued, which can enhance their overall experience with your brand.\n\n2. **Reduced Effort**: Each subsequent contact a customer has to make to resolve an issue increases their effort. This can lead to frustration and dissatisfaction. By solving it the first time, you're min

In [101]:
display_markdown("**Statistical Significance (without Vs with RAG generations)**", raw= True)
print_stats_significance(numeric_scores(without_rag_responses[model_id]), numeric_scores(rag_responses[model_id]), "accuracy without RAG", "accuracy with RAG")

**Statistical Significance (without Vs with RAG generations)**

granite32-8b
 accuracy without RAG                              :     0.8000
 accuracy with RAG                                 :     0.8000
 p_value                                           :     1.0000

p_value>=0.05 so this result is NOT statistically significant.
You can conclude that there is not enough data to tell which is better.
Note that this data includes 5 questions which typically produces a margin of error of around +/-44.7%.
So the two are probably roughly within that margin of error or so.


## Key Takeaways
This tutorial demonstrates how to evaluate an agentic workflow, without and without RAG tool, using the Llama Stack reference implementation.
We do so by initializing an agent, with optional access to the RAG tool, then invoking the agent evaluation against a predefined reference of sample Q&A. 
Please check out our [complementary tutorial](../rag_agentic/notebooks/Level4_RAG_agent.ipynb) for an agentic RAG example.